In [1]:
import numpy as np
from os import listdir
from os.path import isfile, join
import tifffile
from cellpose import models, io, core
import time
from sklearn.model_selection import train_test_split
from statistics import mean
import torch
from ptflops import get_model_complexity_info

c:\Users\rz200\AppData\Local\anaconda3\envs\cellprob\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = models.CellposeModel(gpu=core.use_gpu(), model_type='cyto', device=torch.device('cuda:0'))

In [5]:
unet = models.UnetModel(gpu=core.use_gpu()).net

In [16]:
input_size = (2, 224, 224) 
flops, params = get_model_complexity_info(unet, input_size)

CPnet(
  3.97 M, 100.000% Params, 9.59 GMac, 100.000% MACs, 
  (downsample): downsample(
    1.17 M, 29.570% Params, 2.59 GMac, 27.045% MACs, 
    (down): Sequential(
      1.17 M, 29.570% Params, 2.59 GMac, 27.015% MACs, 
      (conv_down_0): convdown(
        9.92 k, 0.250% Params, 499.65 MMac, 5.212% MACs, 
        (conv): Sequential(
          9.92 k, 0.250% Params, 499.65 MMac, 5.212% MACs, 
          (conv_0): Sequential(
            612, 0.015% Params, 30.81 MMac, 0.321% MACs, 
            (0): BatchNorm2d(4, 0.000% Params, 200.7 KMac, 0.002% MACs, 2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): ReLU(0, 0.000% Params, 100.35 KMac, 0.001% MACs, inplace=True)
            (2): Conv2d(608, 0.015% Params, 30.51 MMac, 0.318% MACs, 2, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
          (conv_1): Sequential(
            9.31 k, 0.235% Params, 468.84 MMac, 4.891% MACs, 
            (0): BatchNorm2d(64, 0.002% Params, 3.21 MMac,

In [17]:
print(flops)
print(params)

9.59 GMac
3.97 M


In [18]:
with torch.cuda.device(0):
  net = models.UnetModel(gpu=core.use_gpu()).net
  macs, params = get_model_complexity_info(net, (2, 224, 224), as_strings=True,
                                           print_per_layer_stat=True, verbose=True)
  print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
  print('{:<30}  {:<8}'.format('Number of parameters: ', params))

CPnet(
  3.97 M, 100.000% Params, 9.59 GMac, 100.000% MACs, 
  (downsample): downsample(
    1.17 M, 29.570% Params, 2.59 GMac, 27.045% MACs, 
    (down): Sequential(
      1.17 M, 29.570% Params, 2.59 GMac, 27.015% MACs, 
      (conv_down_0): convdown(
        9.92 k, 0.250% Params, 499.65 MMac, 5.212% MACs, 
        (conv): Sequential(
          9.92 k, 0.250% Params, 499.65 MMac, 5.212% MACs, 
          (conv_0): Sequential(
            612, 0.015% Params, 30.81 MMac, 0.321% MACs, 
            (0): BatchNorm2d(4, 0.000% Params, 200.7 KMac, 0.002% MACs, 2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): ReLU(0, 0.000% Params, 100.35 KMac, 0.001% MACs, inplace=True)
            (2): Conv2d(608, 0.015% Params, 30.51 MMac, 0.318% MACs, 2, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
          (conv_1): Sequential(
            9.31 k, 0.235% Params, 468.84 MMac, 4.891% MACs, 
            (0): BatchNorm2d(64, 0.002% Params, 3.21 MMac,

In [20]:
#import one of the models
from u_net import UNet

#load the model
model = UNet()
model = model.to('cuda:0')
model.load_state_dict(torch.load('train_dir/models/distillation_model_SIM_1'))

with torch.cuda.device(0):
  net = model
  macs, params = get_model_complexity_info(net, (1, 224, 224), as_strings=True,
                                           print_per_layer_stat=True, verbose=True)
  print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
  print('{:<30}  {:<8}'.format('Number of parameters: ', params))

UNet(
  116.75 k, 100.000% Params, 1.38 GMac, 100.000% MACs, 
  (encoder): Encoder(
    71.79 k, 61.490% Params, 475.27 MMac, 34.437% MACs, 
    (encBlocks): ModuleList(
      71.79 k, 61.490% Params, 473.86 MMac, 34.336% MACs, 
      (0): Block(
        2.48 k, 2.124% Params, 125.24 MMac, 9.075% MACs, 
        (conv1): Conv2d(160, 0.137% Params, 8.03 MMac, 0.582% MACs, 1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(0, 0.000% Params, 802.82 KMac, 0.058% MACs, )
        (conv2): Conv2d(2.32 k, 1.987% Params, 116.41 MMac, 8.435% MACs, 16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (1): Block(
        13.89 k, 11.895% Params, 174.61 MMac, 12.652% MACs, 
        (conv1): Conv2d(4.64 k, 3.974% Params, 58.2 MMac, 4.217% MACs, 16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(0, 0.000% Params, 401.41 KMac, 0.029% MACs, )
        (conv2): Conv2d(9.25 k, 7.921% Params, 116.01 MMac, 8.406% MACs, 32, 32, kerne